<a href="https://colab.research.google.com/github/aycaecemgul/ImageProcessingProject2/blob/master/dog_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import numpy as np
from numpy import asarray
import PIL
from PIL import Image
import skimage
from skimage.transform import resize,rotate
from skimage.color import rgb2gray,gray2rgb
from skimage.filters import median,sato
from skimage.exposure import equalize_adapthist
from matplotlib import pyplot as plt
from skimage.io import imread
import cv2 as cv

In [3]:
#CONNECT TO DRIVE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
DIR="/content/drive/MyDrive/Colab Notebooks/images"
IMG_SIZE = 120

In [ ]:
CATEGORIES=["Afghan Hound","Beagle","Blenheim Spaniel","Japanese Spaniel","Maltese Terrier",
            "Papillon","Pom","Rhodesian Ridgeback","Samoyed","Shih-Tzu"]

In [ ]:
#FIXING IMAGE SIZE TO 120x120
for category in os.listdir(DIR):
    path = os.path.join(DIR, category)
    for img in os.listdir(path):
        filename = os.path.join(path, img)
        img_array = asarray(Image.open(filename).convert('RGB'))
        img_array = resize(img_array, (IMG_SIZE, IMG_SIZE))
        plt.imsave(filename,img_array)

In [ ]:
#counts to total number of images
i=0
for category in os.listdir(DIR):
    path = os.path.join(DIR, category)
    for img in os.listdir(path):
      i+=1

print("amount of images= "+str(i))

amount of images= 2014


In [7]:
training_data=[]

In [8]:
for category in os.listdir(DIR):
    path = os.path.join(DIR, category)
    for img in os.listdir(path):
        class_no = os.listdir(DIR).index(category)
        filename = os.path.join(path, img)
        img_array = asarray(skimage.io.imread(filename))
        training_data.append([img_array, class_no])
        #data augmentation
        training_data.append([cv.flip(img_array,0), class_no]) 
        training_data.append([rotate(img_array, angle=30, resize=False), class_no])
        training_data.append([cv.flip(img_array,1), class_no]) 
        training_data.append([rotate(img_array, angle=60, resize=False), class_no])

In [10]:
X=[]
y=[]

for image,label in training_data:
    X.append(image)
    y.append(label)

In [11]:
X=np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE,3)/255.0
y = np.array(y)


In [12]:
X.shape


(10065, 120, 120, 3)

In [13]:
import pickle

In [14]:
pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [15]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

In [16]:
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
import sklearn
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,shuffle= True)
X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.33, shuffle= True )

In [18]:
base_model = InceptionV3(
                                weights='imagenet',
                                include_top=False,
                                input_shape=(IMG_SIZE, IMG_SIZE,3)
                                )

base_model.trainable=False

model = tf.keras.Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

87916544/87910968 [==============================] - 1s 0us/step


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 2, 2, 2048)        21802784  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                81930     
Total params: 21,884,714
Trainable params: 81,930
Non-trainable params: 21,802,784
_________________________________________________________________


In [20]:
model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

In [21]:
aug = ImageDataGenerator()
model.fit(aug.flow(X_train, y_train), validation_data = (X_val, y_val), epochs = 15,verbose=2)

Epoch 1/15
142/142 - 152s - loss: 3.3018 - accuracy: 0.3810 - val_loss: 2.7526 - val_accuracy: 0.4353
Epoch 2/15
142/142 - 145s - loss: 1.9775 - accuracy: 0.4937 - val_loss: 2.4788 - val_accuracy: 0.4569
Epoch 3/15
142/142 - 145s - loss: 1.6489 - accuracy: 0.5362 - val_loss: 2.7578 - val_accuracy: 0.4420
Epoch 4/15
142/142 - 145s - loss: 1.4724 - accuracy: 0.5676 - val_loss: 3.3428 - val_accuracy: 0.4344
Epoch 5/15
142/142 - 145s - loss: 1.3675 - accuracy: 0.5935 - val_loss: 2.8448 - val_accuracy: 0.4753
Epoch 6/15
142/142 - 146s - loss: 1.0576 - accuracy: 0.6369 - val_loss: 2.8797 - val_accuracy: 0.4712
Epoch 7/15
142/142 - 146s - loss: 1.0642 - accuracy: 0.6338 - val_loss: 3.0357 - val_accuracy: 0.4793
Epoch 8/15


KeyboardInterrupt: ignored

In [ ]:
model.save("dog_classifier.model")

In [ ]:
the_model=tf.keras.models.load_model("dog_classifier.model")